In [1]:
# R libraries: Seurat, SeuratObject, zellkonverter, basilisk
# Read in RDS file into R: sce_nn <- Seurat::as.SingleCellExperiment(SKS_All_nncNeu_CTsc05_integrated,assay='RNA')
# Convert to SingleCellExperiment (SCE) Object: sce_nn <- Seurat::as.SingleCellExperiment(SKS_All_nncNeu_CTsc05_integrated,assay='RNA')
# Write SCE object to h5ad file: writeH5AD(sce_nn,'nn.h5ad')

import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gseapy
from scvi_colab import install
install()
run_outside_colab=True
import scvi
import scipy.sparse
import decoupler as dc
import torch
import gzip
from gtfparse import read_gtf
from pybiomart import Dataset,Server

scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

from rds2py import as_summarized_experiment, read_rds

/opt/applications/python/3.11.4/gnu/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/opt/applications/python/3.11.4/gnu/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/opt/applications/python/3.11.4/gnu/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/opt/applications/python/3.11.4/gnu/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/opt/applications/python/3.11.4/gnu/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: 

ModuleNotFoundError: No module named 'pybiomart'

In [2]:
inh_v2 = sc.read_h5ad('../../HIV_data/single_cell_scorch/inh_v2.h5ad')
exc_v2 = sc.read_h5ad('../../HIV_data/single_cell_scorch/exc_v2.h5ad')
glia_v2 = sc.read_h5ad('../../HIV_data/single_cell_scorch/glia_v2.h5ad')
nn_v2 = sc.read_h5ad('../../HIV_data/single_cell_scorch/nn_v2.h5ad')

In [3]:
pd.unique(inh_v2.obs.study)

['Sestan', 'Spudich', 'Kellis']
Categories (3, object): ['Kellis', 'Sestan', 'Spudich']

In [4]:
pd.unique(exc_v2.obs.study)

['Sestan', 'Spudich', 'Kellis']
Categories (3, object): ['Kellis', 'Sestan', 'Spudich']

In [5]:
pd.unique(glia_v2.obs.study)

['Sestan', 'Spudich', 'Kellis']
Categories (3, object): ['Kellis', 'Sestan', 'Spudich']

In [6]:
pd.unique(nn_v2.obs.study)

['Sestan', 'Spudich', 'Kellis']
Categories (3, object): ['Kellis', 'Sestan', 'Spudich']

In [7]:
inh_v2.obs

,orig.ident,nCount_RNA,nFeature_RNA,sample,percent.mt,nCount_SCT,nFeature_SCT,SCT_snn_res.0.5,seurat_clusters,study,subtype,subclass,class,TopSubclassScore,TopClassScore,TopSubtypeScore,integrated_snn_res.0.5,ident
HSB106_1_AAACCCAGTTGCTCGG-1,SeuratProject,13193.0,5057,HSB106_1,1.099068,3189.0,1857,13,21,Sestan,InN LHX6 HGF STON2,PVALB,InN,0.548885,0.968450,0.478508,21,21
HSB106_1_AAACGAAAGGTGCATG-1,SeuratProject,6099.0,3305,HSB106_1,0.705034,4166.0,3037,7,19,Sestan,InN VIP FREM2,VIP,InN,0.993530,0.993530,0.583200,19,19
HSB106_1_AAAGGGCAGGTCGAGT-1,SeuratProject,9841.0,4571,HSB106_1,3.221217,3768.0,2636,20,12,Sestan,InN LAMP5 BMP7,LAMP5 RELN,InN,0.836297,0.995448,0.803892,12,12
HSB106_1_AAAGGTAGTGTGGACA-1,SeuratProject,9232.0,4464,HSB106_1,0.660745,4007.0,2905,13,21,Sestan,InN PVALB PRKCH,PVALB,InN,0.927638,0.981637,0.429288,21,21
HSB106_1_AAAGTCCGTGCCAAGA-1,SeuratProject,8881.0,4027,HSB106_1,0.709380,3969.0,2684,7,23,Sestan,InN ADARB2 FAM19A1,ADARB2 KCNG1,InN,0.951289,0.993356,0.544453,23,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NIH1308_08_BA9-FC_TTTCAGTTCAATGACC-1,SeuratProject,5645.0,2494,NIH1308_08_BA9-FC,1.293180,4251.0,2477,6,3,Kellis,InN SST TRPC7,SST,InN,0.991082,0.991082,0.644455,3,3
NIH1308_08_BA9-FC_TTTCGTCCATCTTGAG-1,SeuratProject,12844.0,4048,NIH1308_08_BA9-FC,1.744005,3762.0,1933,6,6,Kellis,InN SST FREM1,SST,InN,0.982054,0.984561,0.655625,6,6
NIH1308_08_BA9-FC_TTTCTCACAAATTCGT-1,SeuratProject,13027.0,3967,NIH1308_08_BA9-FC,0.514316,3735.0,1827,9,7,Kellis,InN VIP MEGF11 LHFP,VIP,InN,0.991385,0.991385,0.720375,7,7
NIH1308_08_BA9-FC_TTTGCGGAGGAAGTAT-1,SeuratProject,11602.0,4214,NIH1308_08_BA9-FC,1.111877,3828.0,2188,9,24,Kellis,InN VIP CLSTN2,VIP,InN,0.945951,0.998653,0.396605,24,24


In [8]:
set(inh_v2.obs.iloc[np.where(
    (inh_v2.obs.study=='Kellis')
)[0],3])

{'HCCWQ_09_BA9-FC',
 'HCcFW_07_BA9-FC',
 'HCcHO_05_BA9-FC',
 'HCcLS_01_BA9-FC',
 'HCcOE_06_BA9-FC',
 'HCcOV_03_BA9-FC',
 'HCcPN_02_BA9-FC',
 'HCcWK_12_BA9-FC',
 'HCtBA_23_BA9-FC',
 'HCtDI_24_BA9-FC',
 'HCtDL_13_BA9-FC',
 'HCtIU_15_BA9-FC',
 'HCtPA_16_BA9-FC',
 'HCtQQ_19_BA9-FC',
 'HCtRP_17_BA9-FC',
 'HCtVE_20_BA9-FC',
 'HCtVF_21_BA9-FC',
 'HCtWL_22_BA9-FC',
 'HccUD_10_BA9-FC',
 'HctLI_14_BA9-FC',
 'NIH1308_08_BA9-FC'}

In [9]:
set(exc_v2.obs.iloc[np.where(
    (exc_v2.obs.study=='Kellis')
)[0],3])

{'HCCWQ_09_BA9-FC',
 'HCTXC_18_BA9-FC',
 'HCcFW_07_BA9-FC',
 'HCcHO_05_BA9-FC',
 'HCcLS_01_BA9-FC',
 'HCcOE_06_BA9-FC',
 'HCcOV_03_BA9-FC',
 'HCcPN_02_BA9-FC',
 'HCcWK_12_BA9-FC',
 'HCtBA_23_BA9-FC',
 'HCtDI_24_BA9-FC',
 'HCtDL_13_BA9-FC',
 'HCtIU_15_BA9-FC',
 'HCtPA_16_BA9-FC',
 'HCtQQ_19_BA9-FC',
 'HCtRP_17_BA9-FC',
 'HCtVE_20_BA9-FC',
 'HCtVF_21_BA9-FC',
 'HCtWL_22_BA9-FC',
 'HccUD_10_BA9-FC',
 'HctLI_14_BA9-FC',
 'NIH1308_08_BA9-FC'}

In [10]:
set(glia_v2.obs.iloc[np.where(
    (glia_v2.obs.study=='Kellis')
)[0],3])

{'HCCWQ_09_BA9-FC',
 'HCTXC_18_BA9-FC',
 'HCcFW_07_BA9-FC',
 'HCcHO_05_BA9-FC',
 'HCcLS_01_BA9-FC',
 'HCcOE_06_BA9-FC',
 'HCcOV_03_BA9-FC',
 'HCcPN_02_BA9-FC',
 'HCcWK_12_BA9-FC',
 'HCtBA_23_BA9-FC',
 'HCtDI_24_BA9-FC',
 'HCtDL_13_BA9-FC',
 'HCtIU_15_BA9-FC',
 'HCtPA_16_BA9-FC',
 'HCtQQ_19_BA9-FC',
 'HCtRP_17_BA9-FC',
 'HCtVE_20_BA9-FC',
 'HCtVF_21_BA9-FC',
 'HCtWL_22_BA9-FC',
 'HccUD_10_BA9-FC',
 'HctLI_14_BA9-FC',
 'NIH1308_08_BA9-FC'}

In [11]:
set(nn_v2.obs.iloc[np.where(
    (nn_v2.obs.study=='Kellis')
)[0],3])

{'HCtPA_16_BA9-FC', 'HCtRP_17_BA9-FC', 'HctLI_14_BA9-FC'}

In [12]:
set(inh_v2.obs.iloc[np.where(
    (inh_v2.obs.study=='Spudich')
)[0],3])

{'10280_HIV_PFC',
 '10288_HIV_PFC',
 '10304_HIV_PFC',
 '10305_HIV_PFC',
 '1068_HIV_PFC',
 '1129_HIV_PFC',
 '1137_HIV_PFC',
 '6800127569_HIVOUD_PFC_MAH',
 '6800706867_HIVOUD_PFC_MAH',
 '6801066772_HIV_PFC_MAH',
 '6801187468_HIV_PFC_MAH',
 '7100518287_HIV_PFC_MAH',
 '7101847783_HIV_PFC_MAH',
 '7102096765_HIV_PFC_MAH',
 '7200776574_HIV_PFC_MAH',
 'HCCCX_OUD_PFC',
 'HCCCY_CTR_PFC',
 'HCCDE_OUD_PFC_MAH',
 'HCCRS_CTR_PFC',
 'HCCX_OUD_PFC_MAH',
 'HCTGD_CTR_PFC',
 'HCTKN_CTR_PFC_MAH',
 'HCTLI_CTR_PFC',
 'HCTMU_CTR_PFC',
 'HCTMW_CTR_PFC_MAH',
 'HCTPK_CTR_PFC',
 'HCTTS_CTR_PFC_MAH',
 'HCTXJ_CTR_PFC',
 'HCcPL_CTR_PFC_MAH',
 'HCtME_CTR_PFC_MAH',
 'HCtNZ_CTR_PFC_MAH'}

In [13]:
set(exc_v2.obs.iloc[np.where(
    (exc_v2.obs.study=='Spudich')
)[0],3])

{'10280_HIV_PFC',
 '10288_HIV_PFC',
 '10304_HIV_PFC',
 '10305_HIV_PFC',
 '1068_HIV_PFC',
 '1129_HIV_PFC',
 '1137_HIV_PFC',
 '6800127569_HIVOUD_PFC_MAH',
 '6800706867_HIVOUD_PFC_MAH',
 '6801066772_HIV_PFC_MAH',
 '6801187468_HIV_PFC_MAH',
 '7100518287_HIV_PFC_MAH',
 '7101847783_HIV_PFC_MAH',
 '7102096765_HIV_PFC_MAH',
 '7200776574_HIV_PFC_MAH',
 'HCCCX_OUD_PFC',
 'HCCCY_CTR_PFC',
 'HCCDE_OUD_PFC_MAH',
 'HCCRS_CTR_PFC',
 'HCCX_OUD_PFC_MAH',
 'HCTGD_CTR_PFC',
 'HCTKN_CTR_PFC_MAH',
 'HCTLI_CTR_PFC',
 'HCTMU_CTR_PFC',
 'HCTMW_CTR_PFC_MAH',
 'HCTPK_CTR_PFC',
 'HCTTS_CTR_PFC_MAH',
 'HCTXJ_CTR_PFC',
 'HCcPL_CTR_PFC_MAH',
 'HCtME_CTR_PFC_MAH',
 'HCtNZ_CTR_PFC_MAH'}

In [14]:
set(glia_v2.obs.iloc[np.where(
    (glia_v2.obs.study=='Spudich')
)[0],3])

{'10280_HIV_PFC',
 '10288_HIV_PFC',
 '10304_HIV_PFC',
 '10305_HIV_PFC',
 '1068_HIV_PFC',
 '1129_HIV_PFC',
 '1137_HIV_PFC',
 '6800127569_HIVOUD_PFC_MAH',
 '6800706867_HIVOUD_PFC_MAH',
 '6801066772_HIV_PFC_MAH',
 '6801187468_HIV_PFC_MAH',
 '7100518287_HIV_PFC_MAH',
 '7101847783_HIV_PFC_MAH',
 '7102096765_HIV_PFC_MAH',
 '7200776574_HIV_PFC_MAH',
 'HCCCX_OUD_PFC',
 'HCCCY_CTR_PFC',
 'HCCDE_OUD_PFC_MAH',
 'HCCRS_CTR_PFC',
 'HCCX_OUD_PFC_MAH',
 'HCTGD_CTR_PFC',
 'HCTKN_CTR_PFC_MAH',
 'HCTLI_CTR_PFC',
 'HCTMU_CTR_PFC',
 'HCTMW_CTR_PFC_MAH',
 'HCTPK_CTR_PFC',
 'HCTTS_CTR_PFC_MAH',
 'HCTXJ_CTR_PFC',
 'HCcPL_CTR_PFC_MAH',
 'HCtME_CTR_PFC_MAH',
 'HCtNZ_CTR_PFC_MAH'}

In [15]:
set(nn_v2.obs.iloc[np.where(
    (nn_v2.obs.study=='Spudich')
)[0],3])

{'10280_HIV_PFC',
 '10288_HIV_PFC',
 '10304_HIV_PFC',
 '10305_HIV_PFC',
 '1137_HIV_PFC',
 '6800127569_HIVOUD_PFC_MAH',
 '6800706867_HIVOUD_PFC_MAH',
 '6801066772_HIV_PFC_MAH',
 '6801187468_HIV_PFC_MAH',
 '7100518287_HIV_PFC_MAH',
 '7101847783_HIV_PFC_MAH',
 '7102096765_HIV_PFC_MAH',
 '7200776574_HIV_PFC_MAH',
 'HCCCX_OUD_PFC',
 'HCCCY_CTR_PFC',
 'HCCDE_OUD_PFC_MAH',
 'HCCRS_CTR_PFC',
 'HCCX_OUD_PFC_MAH',
 'HCTGD_CTR_PFC',
 'HCTKN_CTR_PFC_MAH',
 'HCTLI_CTR_PFC',
 'HCTMU_CTR_PFC',
 'HCTMW_CTR_PFC_MAH',
 'HCTPK_CTR_PFC',
 'HCTTS_CTR_PFC_MAH',
 'HCTXJ_CTR_PFC',
 'HCcPL_CTR_PFC_MAH',
 'HCtME_CTR_PFC_MAH',
 'HCtNZ_CTR_PFC_MAH'}

In [16]:
set(inh_v2.obs.iloc[np.where(
    (inh_v2.obs.study=='Sestan')
)[0],3])

{'HSB106_1',
 'HSB106_2',
 'HSB106_3',
 'HSB106_4',
 'HSB189_1',
 'HSB189_2',
 'HSB189_3',
 'HSB189_4',
 'HSB340_1',
 'HSB340_2',
 'HSB340_3',
 'HSB340_4',
 'HSB628_1',
 'HSB628_2',
 'HSB628_3',
 'HSB628_4'}

In [17]:
set(exc_v2.obs.iloc[np.where(
    (exc_v2.obs.study=='Sestan')
)[0],3])

{'HSB106_1',
 'HSB106_2',
 'HSB106_3',
 'HSB106_4',
 'HSB189_1',
 'HSB189_2',
 'HSB189_3',
 'HSB189_4',
 'HSB340_1',
 'HSB340_2',
 'HSB340_3',
 'HSB340_4',
 'HSB628_1',
 'HSB628_2',
 'HSB628_3',
 'HSB628_4'}

In [18]:
set(glia_v2.obs.iloc[np.where(
    (glia_v2.obs.study=='Sestan')
)[0],3])

{'HSB106_1',
 'HSB106_2',
 'HSB106_3',
 'HSB106_4',
 'HSB189_1',
 'HSB189_2',
 'HSB189_3',
 'HSB189_4',
 'HSB340_1',
 'HSB340_2',
 'HSB340_3',
 'HSB340_4',
 'HSB628_1',
 'HSB628_2',
 'HSB628_3',
 'HSB628_4'}

In [19]:
set(nn_v2.obs.iloc[np.where(
    (nn_v2.obs.study=='Sestan')
)[0],3])

{'HSB106_1',
 'HSB106_2',
 'HSB106_3',
 'HSB106_4',
 'HSB189_1',
 'HSB189_2',
 'HSB189_3',
 'HSB189_4',
 'HSB340_1',
 'HSB340_2',
 'HSB340_3',
 'HSB340_4',
 'HSB628_1',
 'HSB628_2',
 'HSB628_3',
 'HSB628_4'}

In [2]:
scorch_meta_xl = pd.read_excel('../../HIV_data/single_cell_scorch/SCORCHFormatSubjectMetadataDec2023.xlsx',sheet_name=None)

/opt/applications/python/3.11.4/gnu/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
scorch_meta_xl['subject'].columns

Index(['subject_name', 'subject_source', 'subject_source_id',
       'parent_subject_id', 'subject_source_catalog_number', 'subject_type',
       'species', 'strain_subspecies_name', 'subject_genotype', 'subprogram',
       'project_short_name', 'cohort_id', 'metadata_access',
       'controlled_access_fields', 'sex', 'additional_assays', 'age_value',
       'age_unit', 'subject_event_name', 'subject_comments',
       'postmortem_interval', 'time_of_death', 'autopsy_year', 'transgender',
       'race', 'ethnicity', 'cause_of_death', 'viral_status', 'hiv_strain',
       'years_with_hiv', 'cd4_counts', 'cd4_nadir', 'cognitive_status',
       'brain_pathology', 'spinal_cord_pathology', 'antiretroviral_treatment',
       'plasma_viral_load_measurement',
       'months_since_last_plasma_viral_measurement', 'tox_history_amp',
       'tox_history_bar', 'tox_history_bzo', 'tox_history_bup',
       'tox_history_thc', 'tox_history_coc', 'tox_history_mtd',
       'tox_history_met', 'tox_history_o

In [4]:
scorch_meta_xl = pd.read_excel('../../HIV_data/single_cell_scorch/SCORCHFormatSubjectMetadataSept2024.xlsx',sheet_name=None)
scorch_meta_xl['subject'].columns

Index(['subject_name', 'subject_source', 'subject_source_id',
       'subject_source_catalog_number', 'subject_type', 'species',
       'strain_subspecies_name', 'subject_genotype', 'grant_number',
       'grant_name', 'project_short_name', 'cohort_id', 'metadata_access',
       'controlled_access_fields', 'sex', 'additional_assays', 'age_value',
       'age_unit', 'subject_event_name', 'subject_comments',
       'postmortem_interval', 'time_of_death', 'autopsy_year', 'transgender',
       'race', 'ethnicity', 'cause_of_death', 'viral_status', 'hiv_strain',
       'years_with_hiv', 'cd4_counts', 'cd4_nadir', 'cognitive_status',
       'brain_pathology', 'spinal_cord_pathology', 'antiretroviral_therapy',
       'plasma_viral_load_measurement',
       'months_since_last_plasma_viral_measurement', 'tox_history_amp',
       'tox_history_bar', 'tox_history_bzo', 'tox_history_bup',
       'tox_history_thc', 'tox_history_coc', 'tox_history_mtd',
       'tox_history_met', 'tox_history_opi', 't

In [77]:
kellis_meta = pd.read_csv('../../HIV_data/single_cell_scorch/Kellis_sample_metadata.tsv',sep='\t')

In [87]:
kellis_meta.Subject.str.contains('HCCWQ')

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
Name: Subject, dtype: bool

In [5]:
spudich_hiv = np.where(
    (scorch_meta_xl['subject'].subject_name.str.contains('6801066772')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('6801187468')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('7100518287')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('7101847783')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('7102096765')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('7200776574')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('10280')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('10288')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('10304')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('10305')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('1068')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('1129')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('1137')) 
    
)[0]
spudich_controls = np.where(
    (scorch_meta_xl['subject'].subject_name.str.contains('HCCCY')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCCRS')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTGD')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTKN')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTLI')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTMU')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTMW')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTPK')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTTS')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCTXJ')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCcPL')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCtME')) |
    (scorch_meta_xl['subject'].subject_name.str.contains('HCtNZ')) 
)[0]

In [116]:
kellis_samps=np.where(
    (kellis_meta['Subject'].str.contains('HCCWQ')) |
    (kellis_meta['Subject'].str.contains('HCTXS')) |
    (kellis_meta['Subject'].str.contains('HCcFW')) |
    (kellis_meta['Subject'].str.contains('HCcHO')) |
    (kellis_meta['Subject'].str.contains('HCcLS')) |
    (kellis_meta['Subject'].str.contains('HCcOE')) |
    (kellis_meta['Subject'].str.contains('HCcOV')) |
    (kellis_meta['Subject'].str.contains('HCcPN')) |
    (kellis_meta['Subject'].str.contains('HCcWK')) |
    (kellis_meta['Subject'].str.contains('HCtBA')) |
    (kellis_meta['Subject'].str.contains('HCtDI')) |
    (kellis_meta['Subject'].str.contains('HCtDL')) |
    (kellis_meta['Subject'].str.contains('HCtIU')) |
    (kellis_meta['Subject'].str.contains('HCtPA')) |
    (kellis_meta['Subject'].str.contains('HCtQQ')) |
    (kellis_meta['Subject'].str.contains('HCtRP')) |
    (kellis_meta['Subject'].str.contains('HCtVE')) |
    (kellis_meta['Subject'].str.contains('HCtVF')) |
    (kellis_meta['Subject'].str.contains('HCtWL')) |
    (kellis_meta['Subject'].str.contains('HccUD')) |
    (kellis_meta['Subject'].str.contains('HctLI')) |
    (kellis_meta['Subject'].str.contains('NIH1308')) 
)[0]

In [ ]:
 'HCCWQ_09_BA9-FC',
 'HCTXC_18_BA9-FC',
 'HCcFW_07_BA9-FC',
 'HCcHO_05_BA9-FC',
 'HCcLS_01_BA9-FC',
 'HCcOE_06_BA9-FC',
 'HCcOV_03_BA9-FC',
 'HCcPN_02_BA9-FC',
 'HCcWK_12_BA9-FC',
 'HCtBA_23_BA9-FC',
 'HCtDI_24_BA9-FC',
 'HCtDL_13_BA9-FC',
 'HCtIU_15_BA9-FC',
 'HCtPA_16_BA9-FC',
 'HCtQQ_19_BA9-FC',
 'HCtRP_17_BA9-FC',
 'HCtVE_20_BA9-FC',
 'HCtVF_21_BA9-FC',
 'HCtWL_22_BA9-FC',
 'HccUD_10_BA9-FC',
 'HctLI_14_BA9-FC',
 'NIH1308_08_BA9-FC'

In [28]:
kellis

array([268])

In [26]:
scorch_meta_xl['subject'].loc[spudich_hiv,['cd4_counts','cognitive_status','brain_pathology', 'spinal_cord_pathology', 
                                          'antiretroviral_therapy','sex','viral_status']]

,cd4_counts,cognitive_status,brain_pathology,spinal_cord_pathology,antiretroviral_therapy,sex,viral_status
46,400,Unable to reliably assign neurocognitive diagn...,"Atherosclerosis of major brain arteries, Neuro...","Descending fiber tract degeneration, Gracile t...",DRV; RGV; RTV; TMC,male,HIV_positive
48,598,Probable Mild Neurocognitive disorder,"Atherosclerosis of major brain arteries, Hemor...",Spinal cord normal,3TC; ABC; DTG,female,HIV_positive
52,305,Neuropsychological Impairment - Other cause,Arteriolar wall thickening suggestive of chron...,Spinal cord normal,BIK,female,HIV_positive
53,172,Neuropsychological Impairment - Other cause,Arteriolar wall thickening suggestive of chron...,Spinal cord normal,DTG; TRU,male,HIV_positive
86,33,Probable HAD,"Alzheimer type 2 gliosis, Hypoxic or ischemic ...",Spinal cord normal,NONE; NONE,male,HIV_positive
92,60,Neuropsychological Impairment; does not meet c...,Minimal non-diagnostic abnormalities,Spinal cord normal,3TC; ABC; KTA; ZDV; 3TC; ABC; KTA; ZDV,male,HIV_positive
94,not_known,not_known,Alzheimer type 2 gliosis,Spinal cord normal,not_known,male,HIV_positive
133,20,Neuropsychological Impairment or dementia due ...,Atherosclerosis of major brain arteries,Spinal cord normal,ATV; TRU; ATV; TRU; ATR; ATR,male,HIV_positive
134,20,not_known,"Bacterial parenchymal infection, Focal (territ...",Spinal cord normal,EPZ; FPV; RTV; EPZ; FPV; RTV; DRV; TRU; DRV; T...,male,HIV_positive
137,11,Test not done,"Focal (territorial) infarct, Other non-infecti...",Gracile tract atrophy (ascending tract degener...,NONE; NONE,male,HIV_positive


In [22]:
np.mean(pd.to_numeric(scorch_meta_xl['subject'].loc[spudich_hiv,'age_value']))

53.12857142857142

In [23]:
np.mean(pd.to_numeric(scorch_meta_xl['subject'].loc[spudich_controls,'age_value']))

37.5

In [28]:
scorch_meta_xl['subject'].loc[spudich_hiv,['cd4_counts','cognitive_status','brain_pathology', 'spinal_cord_pathology', 
                                          'antiretroviral_therapy','sex','viral_status']].reset_index(drop=True).to_excel('../../results/SCORCH_HIV/hiv-pos_metadata.xlsx')

In [24]:
scorch_meta_xl['subject'].loc[spudich_controls,['subject_name','cd4_counts','cognitive_status','brain_pathology', 'spinal_cord_pathology', 
                                                'antiretroviral_therapy','age_value','sex','viral_status']]

,subject_name,cd4_counts,cognitive_status,brain_pathology,spinal_cord_pathology,antiretroviral_therapy,age_value,sex,viral_status
315,HCCRS,not_applicable,not_known,not_known,not_known,not_applicable,49,female,presumed_none
330,HCTGD,not_applicable,not_known,not_known,not_known,not_applicable,46,male,presumed_none
334,HCTLI,not_applicable,not_known,not_known,not_known,not_applicable,51,male,presumed_none
340,HCTMU,not_applicable,not_known,not_known,not_known,not_applicable,20,male,presumed_none
341,HCTMW,not_applicable,not_known,not_known,not_known,not_applicable,35,male,presumed_none
352,HCTXJ,not_applicable,not_known,not_known,not_known,not_applicable,18,male,presumed_none
366,HCTPK,not_applicable,not_known,not_known,not_known,not_applicable,44,female,NaN
371,HCTTS,not_applicable,not_known,not_known,not_known,not_applicable,43,female,NaN
375,HCTKN,not_applicable,not_known,not_known,not_known,not_applicable,50,male,NaN
378,HCCCY,not_applicable,not_known,not_known,not_known,not_applicable,19,male,NaN


In [107]:
scorch_meta_xl['subject'].loc[kellis,['subject_name','cognitive_status','brain_pathology', 'spinal_cord_pathology', 
                                                'antiretroviral_treatment','age_value','sex','viral_status']]

,subject_name,cognitive_status,brain_pathology,spinal_cord_pathology,antiretroviral_treatment,age_value,sex,viral_status
268,HCCWQ,not_known,not_known,not_known,not_applicable,32,male,HIV_negative


In [117]:
kellis_meta.loc[kellis_samps,:].reset_index(drop=True)

,Subject,Region,COD,Age,Race,Ethnicity,Gender,PMI
0,HCcLS,BA9-FC Nucleus Accumbens,Opioid,39,W,Non-hispanic,F,26
1,HCcPN,BA9-FC Nucleus Accumbens,Opioid,29,W,Hispanic,M,29
2,HCcOV,BA9-FC Nucleus Accumbens,Opioid,30,W,Hispanic,M,30.5
3,HCcHO,BA9-FC Nucleus Accumbenss,Opioid,41,B,Non-hispanic,M,22
4,HCcOE,BA9-FC Nucleus Accumbens,Opioid,42,W,Non-hispanic,M,28.5
5,HCcFW,BA9-FC Nucleus Accumbens,Opioid,59,W,Hispanic,M,24
6,NIH1308,BA9-FC Nucleus Accumbens,Opioid,32,B,Non-hispanic,M,50
7,HCCWQ,BA9-FC Nucleus Accumbens,Opioid,32,W,Hispanic,M,18.8
8,HccUD,BA9-FC Nucleus Accumbens,Opioid,38,W,Non-hispanic,M,19
9,HCcWK,BA9-FC Nucleus Accumbens,Opioid,27,W,Hispanic,M,25


In [123]:
len(set(glia_v2.obs.iloc[np.where(glia_v2.obs.study=='Kellis')[0],3]))#.intersection(kellis_meta.loc[kellis_samps,:].reset_index(drop=True).Subject)

22

In [153]:
set(glia_v2.obs.iloc[np.where(glia_v2.obs.study=='Kellis')[0],3])

{'HCCWQ_09_BA9-FC',
 'HCTXC_18_BA9-FC',
 'HCcFW_07_BA9-FC',
 'HCcHO_05_BA9-FC',
 'HCcLS_01_BA9-FC',
 'HCcOE_06_BA9-FC',
 'HCcOV_03_BA9-FC',
 'HCcPN_02_BA9-FC',
 'HCcWK_12_BA9-FC',
 'HCtBA_23_BA9-FC',
 'HCtDI_24_BA9-FC',
 'HCtDL_13_BA9-FC',
 'HCtIU_15_BA9-FC',
 'HCtPA_16_BA9-FC',
 'HCtQQ_19_BA9-FC',
 'HCtRP_17_BA9-FC',
 'HCtVE_20_BA9-FC',
 'HCtVF_21_BA9-FC',
 'HCtWL_22_BA9-FC',
 'HccUD_10_BA9-FC',
 'HctLI_14_BA9-FC',
 'NIH1308_08_BA9-FC'}

In [154]:
set(kellis_meta.loc[kellis_samps,:].reset_index(drop=True).Subject)

{'HCCWQ',
 'HCcFW ',
 'HCcHO',
 'HCcLS',
 'HCcOE',
 'HCcOV',
 'HCcPN',
 'HCcWK',
 'HCtDI',
 'HCtDL',
 'HCtIU',
 'HCtPA',
 'HCtQQ',
 'HCtRP',
 'HCtVE',
 'HCtVF',
 'HCtWL',
 'HccUD',
 'NIH1308'}